In [2]:
import os
import pandas as pd
import sys
from xbbg import blp
sys.path.append('../src')
import utils

etf_ticker = 'EFA'
asof = '2025-09-25'

hold_df = pd.read_csv(f'cached_holdings/{etf_ticker}_{asof}.csv')

In [3]:
filtered = hold_df[hold_df['adr'].notnull() & (hold_df['adr_exchange'] != 'OOTC') & (hold_df['adr_turnover'] > 5e6)].copy()

In [4]:
filtered.loc[filtered['country'].isin(['BRITAIN','HONG KONG','AUSTRALIA']),'index_future_bbg'] = 'Z'
filtered.loc[filtered['country'].isin(['JAPAN']),'index_future_bbg'] = 'NH'
filtered.loc[filtered['index_future_bbg'].isna(),'index_future_bbg'] = 'VG'


In [5]:
filtered.loc[filtered['country'].isin(['BRITAIN','HONG KONG']),'market_etf_hedge'] = 'EWU'
filtered.loc[filtered['country'].isin(['JAPAN']),'market_etf_hedge'] = 'EWJ'
filtered.loc[filtered['country'].isin(['AUSTRALIA']),'market_etf_hedge'] = 'EWA'
filtered.loc[filtered['market_etf_hedge'].isna(),'market_etf_hedge'] = 'VGK'

In [6]:
filtered[filtered['currency'].isin(['GBp','EUR','JPY','CHF','AUD','DKK'])].to_csv('../data/raw/all_adr_filtered.csv', index=False)
filtered[filtered['currency'].isin(['GBp','EUR','JPY'])].to_csv('../data/raw/adr_top_three_currencies.csv', index=False)
filtered[filtered['currency'].isin(['GBp','EUR'])].to_csv('../data/raw/adr_gbp_eur.csv', index=False)


In [8]:
filtered[filtered['currency'].isin(['GBp','EUR'])]['adr_turnover'].sum()/filtered[filtered['currency'].isin(['GBp','EUR','JPY','CHF','AUD','DKK'])]['adr_turnover'].sum()

0.7655713455345836